In [72]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy
from sqlalchemy import create_engine
import calendar

In [73]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [ ]:
conn_string_cloud = conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='',
                                                                     host='10.247.204.254',
                                                                     port='5433',
                                                                     db='test_vow4'
                                                                     )
engine2 = create_engine(conn_string_cloud, paramstyle='format', isolation_level="READ UNCOMMITTED")
con_test_qry = pd.read_sql_query('Select * from vm_vh_class',con=engine2)
if con_test_qry.shape == (70,5):
    print("Connection Established")
else:
    print("Error in Connection")

In [3]:
path = 'D:/SK/'
state = 'SK'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']==state]
slab_new = slab_new[slab_new['vh_class'].notnull()]

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')
slab_new.reset_index(drop=True, inplace=True)

In [7]:
slab_new.reset_index(drop=True, inplace=True)
#slab_new.loc[:4,'date_from']
slab_new.head

<bound method NDFrame.head of      class_type  date_from    date_to                                descr  \
0             2 2007-01-07 2011-01-31                      MOTOR CYCLE TAX   
1             2 2007-01-07 2011-01-31                      MOTOR CYCLE TAX   
2             2 2007-01-07 2011-01-31                      MOTOR CYCLE TAX   
3             2 2007-01-07 2011-01-31                       MOTOR CARS TAX   
4             1 2007-01-07 2011-01-31  OMNIBUS (EDUCATIONAL INSTITUTE BUS)   
..          ...        ...        ...                                  ...   
308           1 2011-01-02 2099-12-31         LMV (COMMERCIAL/TAXIS/GOODS)   
309           1 2011-01-02 2099-12-31         LMV (COMMERCIAL/TAXIS/GOODS)   
310           1 2011-01-02 2099-12-31         LMV (COMMERCIAL/TAXIS/GOODS)   
311           2 2011-01-02 2099-12-31                      SPECIAL VEH TAX   
312           2 2011-01-02 2099-12-31                      SPECIAL VEH TAX   

     emp_cd             op_dt  pu

In [8]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')
slab_add_new = slab_add_new[slab_add_new['state_cd']==state]

In [9]:
vt_tax_based_on = pd.read_csv(path+'/vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')
#vt_tax_based_on = pd.read_sql_query('Select * from vt_tax_based_on where state_cd ='+"'{}'".format(state),con=engine2)

In [10]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')


In [11]:
vt_owner = pd.read_csv(path+'/vt_owner.csv')
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

In [12]:
vt_permit = pd.read_csv(path+'/vt_permit.csv')
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

In [13]:
tm_office = pd.read_csv(path+'/tm_office.csv')
#tm_office.head()
tm_office = tm_office[tm_office['state_cd']==state]

In [20]:
vt_tax = pd.read_csv(path+'vt_tax.csv')
#vt_tax = pd.read_sql_query('Select * from vt_tax where state_cd ='+"'{}'".format(state),con=engine2)
vt_tax = vt_tax[vt_tax['pur_cd']==58]
vt_tax = vt_tax[vt_tax['tax_mode']!='B']
vt_tax=vt_tax[vt_tax['collected_by']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)


In [14]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [49]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Sikkim defaulter all.xls')
#def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list.columns

Index(['S. No', 'Regn No', 'Office', 'Regn Date', 'Vehicle Class',
       'Vehicle Category', 'purpose Code', 'Tax Type', 'Owner Name',
       'Father Name', 'Complete Present Address', 'Mobile No', 'Laden Wt.',
       'UnLaden Wt.', 'Seating Capacity', 'Standing Capacity', 'Fuel Type',
       'Tax From', 'Tax Upto', 'Tax Cleared Upto', 'Tax Arrear From',
       'Tax Arrear UpTo', 'Permit Valid From', 'Permit Valid Upto',
       'Tax Amount in (Rs)', 'Tax Fine in (Rs)', 'Total Tax Amount in (Rs)',
       'Fitness Validity', 'Financer Name', 'Blacklist Status'],
      dtype='object')

In [53]:
def_list.shape

(9289, 30)

In [52]:
def_list['Vehicle Class'].value_counts()

Motor Car                            5092
M-Cycle/Scooter                      1690
Goods Carrier                        1391
Motor Cab                             504
Motorised Cycle (CC > 25cc)           415
Maxi Cab                               73
Ambulance                              32
Luxury Cab                             29
Excavator (Commercial)                 22
Bus                                    18
Earth Moving Equipment                 12
Fire Fighting Vehicle                   2
Educational Institution Bus             2
Cash Van                                2
Motor Cycle/Scooter-Used For Hire       1
Construction Equipment Vehicle          1
Name: Vehicle Class, dtype: int64

In [46]:
def calculate_formula(formula_df,params_info):
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        #print('l',l)
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        #print(c_formula)
        for m in cf_params:
            if m != '<>':
                val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])                
                c_formula = c_formula.replace(str(m),val)
                #print(c_formula,str(m),val)
        c_formula = c_formula.replace(repr('nan'),'-100')
        #print(c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R' and 'nan' not in c_formula.lower():
            cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
            cf_result = cf_result.loc[0,'?column?']
        else:
            cf_result = False
        if cf_result==True:
            #print('Ind',ind,l)
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    return [ind,summary_df]

In [47]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if '-' in v:
                        v = v.replace('-','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
    return c_formula
'''
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula
'''

"\n        else:\n            c_formula = c_formula.replace('<>','!=')\n\n    # Change '=' to '=='\n    if '<=' in c_formula:\n        c_formula=c_formula.replace('<=','@')\n    if '>=' in c_formula:\n        c_formula=c_formula.replace('>=',';')\n    if '!=' in c_formula:\n        c_formula=c_formula.replace('!=','?')\n    if '=' in c_formula:\n        c_formula = c_formula.replace('=','==')\n    if '@' in c_formula:\n        c_formula=c_formula.replace('@','<=')\n    if ';' in c_formula:\n        c_formula=c_formula.replace(';','>=')\n    if '?' in c_formula:\n        c_formula = c_formula.replace('?','!=')\n    return c_formula\n"

In [17]:
def calculate_rates(true_df,params_info):
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print('t_params',t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        
        #print('r_formula',r_formula)
            
        #rate = eval(r_formula.lower())
        rate = pd.read_sql_query(repr('select '+r_formula)[1:-1],con=engine)
        rate = rate.loc[0,'?column?']
        #print(rate)
        rates.append(rate)        
    true_df['Rate']=rates
    return true_df

In [19]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print('TBO shape',tbo_df.shape)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [18]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt,tax_mode):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        #col= col.replace('_',' ').title()
        p_value = ''
        #print(par,code,tab)
        #if tab=='own':
        #    p_date = 'Purchase Dt'
        #else:
        p_date= 'purchase_dt'
        purchase_dt = ot_df[p_date].to_list()[0]
        
        p_valid_date = purchase_dt[:2].isdigit()
        if p_valid_date==False and tab=='tbo':
            #print(p_valid_date,tab)
            purchase_dt = vt_owner.loc[vt_owner['regn_no']==regn_no][p_date].to_list()[0]
        p_valid_date = purchase_dt[:2].isdigit()
        #print(purchase_dt)
        if purchase_dt[:4].isdigit() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%Y-%m-%d')
        elif purchase_dt[3:6].isalpha() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%b-%Y')
        elif p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%m-%Y')
        if par=='vch_age' and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            #print('p_value',p_value)
            p_value = p_value.years
        elif par=='vch_age_months'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='fin_vch_age'and p_valid_date==True:
            #print(datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%d-%m-%Y'))
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        elif par=='payment_date':
            p_value = rcpt_dt
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['Vh Class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='permit_sub_catg':
            if 'pmt_catg' in ot_df.columns:
                p_value = ot_df['pmt_catg'].to_list()[0]
            else:
                p_value = float("nan")
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print('delayed',p_value)
        elif par == 'delay_months':
            p_value = relativedelta(rcpt_dt, tax_from)
            #print('delayed months',p_value)
            if p_value.days>0:
                bal_m = 1
            p_value = p_value.years * 12 + p_value.months + bal_m
            #print(p_value)
        elif p_valid_date==True:
            #if tab=='own':
            #    col= col.replace('_',' ').title()
            p_value= ot_df[col].to_list()[0]
            #print(p_value)
        else:
            p_value=0
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [20]:
def extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['regn_no']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        #print(last_rcpt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            #for j in c :
                #p= j.replace('_',' ').title()
                #tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]
    #print(tbo_df)
    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        #print(tab)
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt,tax_mode)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt,tax_mode)
    #print('Shape',params_info)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other.shape)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            
            #print(null_df)
            #print('Yes!',)
            #print('Other',code,col,par)
            #if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
            if pd.isnull(params_info.loc[params_info['code']==code]['P_value']).to_list()[0]==True:
                if col == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Pmt Type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Permit Type Available')
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                elif col=='tax_amt':
                    p_value = final_rate
                #print('Other',code,col,par)
                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [21]:
date_format = "%d-%b-%Y"
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Q' ,state_cd='AS',off_cd=141,tm='n'):
    rcpt_dt = tax_upto
    #tm='Q'
    if len(vt_owner[vt_owner['regn_no']==regn_no])>0:
        vh_class = vt_owner[vt_owner['regn_no']==regn_no]['vh_class'].to_list()[0]
        new_vch = 'N'
        #print(vh_class)
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
        return 0
        
    #slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    #tax_from = datetime.strptime(tax_from,'%d-%b-%Y')
    #print(vh_class, pur_cd, state_cd,tax_mode, tax_from)
    slab_new1 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_from)]
    #slab_new2 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_upto)] # (slab_new['date_from'] < tax_from) & 
    #print(slab_new1)
    slab_codes = slab_new1['slab_code'].to_list()
    #print(pur_cd,'Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        print(i,'Missed',regn_no,'vh_class',vh_class)
        return 0

    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
    formula_df = formula_df[formula_df['condition_formula']!='0=0']
    #formula_df = formula_df[formula_df['condition_formula']!='1=1']
    #formula_df.columns
    
    #print('formula_df',formula_df)
        
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    #print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    #print(params_info)
    param_info_owner = params_info[params_info['def_table']=='vt_owner']
    #print('Owner',param_info_owner['column - slab_field'].to_list())
    param_info_other = params_info[params_info['def_table']!='vt_owner']
    #print('Other',param_info_other['column - slab_field'].to_list())
    #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
    #print('TBO',param_info_tbo['column - slab_field'].to_list())
    #print('Initial Shape',params_info.shape)
    params_info = extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate)
    #print('Final Shape',params_info.shape)                                                                        
    #if pur_cd ==58:        
    #print(params_info)
    ind = calculate_formula(formula_df,params_info)[0]
    summary_df = calculate_formula(formula_df,params_info)[1]
    #if pur_cd==57:
    #print(summary_df)
    if len(ind)==0:
        print('No True Condition')
        return 0
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    #print('True_df',true_df)

    #print(t_params)
    true_df_t = true_df[(true_df['type_flag']=='T') | (true_df['type_flag']=='M') | (true_df['type_flag']=='X') ]
    
    true_df_r = true_df[true_df['type_flag']=='R']
    true_df_s = true_df[true_df['type_flag']=='S']
    
    true_df_t = calculate_rates(true_df_t,params_info)
    #print('T',true_df_t)
    #print('S',true_df_s)
    #print(rates)
    #print('True_df',true_df_t)

    t_df = true_df_t[(true_df_t['type_flag']=='T')]
    t_df.reset_index(drop=True, inplace=True)               
    #print(t_df)
    if len(t_df)>0:
        #t_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]
        #print('t_rate',t_rate)
        #if len(t_df)>1:
        t_rate = 0
        upt = datetime.strptime('01-Jan-1900','%d-%b-%Y')
        #until = max(slab_new1[slab_new1['slab_code'] in t_df['slab_code'].to_list() ]['date_to'].to_list())
        for t in range(len(t_df)):
            sc = t_df.loc[t,'slab_code']
            f_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_from']
            t_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_to']
            fro = max(f_dt.item(),tax_from,upt)   
            upt = min(t_dt.item(),tax_upto)
            n_days = (upt - fro).days + 1
            if calendar.isleap(fro.year):
               days_in_year = 366
            else:
               days_in_year = 365  
            per_day_rate = t_df.loc[t,'Rate'] /days_in_year
            t_rate = t_rate + per_day_rate *n_days
            #print(pur_cd,t_rate,fro,upt, days_in_year,t_df.loc[t,'Rate'],per_day_rate,n_days)
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df_t['condition_formula'].to_list()))>0:
        #true_df_t = true_df_t[true_df_t['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df_t['type_flag'].to_list()))==0:
            if true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]> true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = t_rate
            else:
                final_rate = true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()
        elif len({'T','X'} - set(true_df_t['type_flag'].to_list()))==0:
            #print('Here')
            final_rate = true_df_t[true_df_t['type_flag']=='X']['Rate'].to_list()
        else:
            final_rate = t_rate
    #else:       
    #    final_rate = true_df['Rate'].to_list()
    #else:
    #    final_rate = 0
    #print(final_rate)
    #tax_mode== 'Q'
    #Module for Surcharge
    if len(true_df_s) > 0 :
        params_info.loc[params_info['code'] == '<58>','P_value']=final_rate
        true_df_s = calculate_rates(true_df_s,params_info)
        #print(true_df_s)
        surcharge = true_df_s[true_df_s['type_flag']=='S']['Rate'].to_list()
        #print(final_rate,surcharge)
        final_rate = final_rate + surcharge[0]
    return final_rate
    '''
    if len(true_df_r) > 0:
        rr_formula = true_df_r[true_df_r['condition_formula']==f]['rate_formula'].to_list()[0]
        ra_formula = condition_form_convert(reb_params,params_info,rr_formula)
        reb_rate_result = eval(ra_formula.lower())              
        final_rate = final_rate - reb_rate_result
         
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode
    '''
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print('n_quart',n_quart,final_rate,final_rate*n_quart)
        final_rate = final_rate * n_quart
    elif tax_mode == 'Y':
        tax_period = relativedelta(tax_upto, tax_from)
        years_y = tax_period.years * 12 
        months_y = math.ceil(tax_period.months + years_y)
        if tax_period.days > 0:
            months_y = months_y+1
        #print((final_rate / 12),months_y)
        if tm == 'y2q':
            months_to_quart = math.ceil(months_y/3)
            final_rate = (final_rate / 4) * months_to_quart
        else:
            final_rate = (final_rate / 12) * months_y
    '''
    

In [71]:
nc_regn_no_list = []
for i in range(0,10):   
    regn_no = def_list['Regn No'].to_list()[i]
    
    state_cd = state
    #print(regn_no,state)
    #off_cd = 1
    vh_class = def_list['Vehicle Class'].to_list()[i]
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    tax_from = def_list['Tax Arrear From'].to_list()[i]
    #print(tax_from)
    tax_from = datetime.strptime(tax_from, date_format)
    
    tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    #tax_upto = datetime.strptime('04-May-2020','%d-%b-%Y')
    #print('Delayed',tax_upto-tax_from)
    c_tax_amt = round(calculate_tax(regn_no,tax_from,tax_upto,tax_mode='Y',state_cd=state))
    #print(c_tax_amt)
    if c_tax_amt == 0:
        #print('Unable to calculate tax')
        nc_regn_no_list.append(regn_no)
    tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    
    tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    if c_tax_amt !=0:
        c_tax_fine = round(calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,tax_mode='Y',state_cd=state))
    #print(tax_fine)
    if tax_amt!=c_tax_amt: 
        print('Mismatch',i,regn_no, vh_class,str(tax_from)[:11],str(tax_upto)[:11],tax_amt,c_tax_amt,c_tax_fine,tax_fine)
    else:
        print(i, regn_no,vh_class,str(tax_from)[:11],str(tax_upto)[:11], tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]
    

Mismatch 0 SK01G1018 M-Cycle/Scooter 2019-01-22  2021-01-21  900 901.0 1804.0 450.0
Mismatch 1 SK01G1019 M-Cycle/Scooter 2019-01-22  2021-01-21  900 901.0 1804.0 450.0
Mismatch 2 SK01G1089 M-Cycle/Scooter 2012-06-08  2020-06-07  2400 2395.0 19121.0 2400.0
Mismatch 3 SK01G2547 M-Cycle/Scooter 2019-07-29  2020-07-28  450 451.0 452.0 nan
Mismatch 4 SK01G2941 M-Cycle/Scooter 2019-06-16  2020-06-15  300 301.0 302.0 nan
Mismatch 5 SK01G2988 M-Cycle/Scooter 2019-07-20  2020-07-19  450 451.0 452.0 nan
Mismatch 6 SK01P0432 M-Cycle/Scooter 2019-05-05  2021-05-04  600 601.0 1204.0 300.0
Mismatch 7 SK01P1023 M-Cycle/Scooter 2018-11-17  2020-11-16  900 901.0 1804.0 450.0
Mismatch 8 SK01P1151 M-Cycle/Scooter 2016-11-17  2020-11-16  1200 1198.0 4782.0 900.0
Mismatch 9 SK01P2166 M-Cycle/Scooter 2019-04-16  2021-04-15  900 901.0 1804.0 450.0


In [67]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Sikkim defaulter all.xls')

In [68]:
print(def_list.shape)
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
#def_list = def_list[def_list['Vehicle Class']=='Motor Cab']
def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
def_list = def_list[def_list['Tax Type']=='MV Tax']
def_list.reset_index(drop=True, inplace=True)
print(def_list.shape)

(26651, 30)
(9289, 30)


In [69]:
def_list['Vehicle Class'].value_counts()

Motor Car                            5092
M-Cycle/Scooter                      1690
Goods Carrier                        1391
Motor Cab                             504
Motorised Cycle (CC > 25cc)           415
Maxi Cab                               73
Ambulance                              32
Luxury Cab                             29
Excavator (Commercial)                 22
Bus                                    18
Earth Moving Equipment                 12
Fire Fighting Vehicle                   2
Educational Institution Bus             2
Cash Van                                2
Motor Cycle/Scooter-Used For Hire       1
Construction Equipment Vehicle          1
Name: Vehicle Class, dtype: int64

In [70]:
def_list = def_list[def_list['Vehicle Class']=='M-Cycle/Scooter']
def_list.shape

(1690, 30)

In [392]:
c_formula = '750+(CEIL((7520-1000) / 1000.00) * 750)'
pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)

,?column?
0,6000.0


In [182]:
slab_new1 = slab_new[(slab_new['vh_class']==86) & (slab_new['pur_cd']==58) & (slab_new['tax_mode']=='Y') &(slab_new['state_cd']=='BR') & (slab_new['date_from'] < tax_from) & (slab_new['date_to'] > tax_from)]

In [242]:
(t_dt - f_dt).item().days

234

In [346]:
vt_owner[vt_owner['regn_no']=='BR01PA6073']['purchase_dt']

246180    02-10-2009
Name: purchase_dt, dtype: object

In [151]:
def_list[def_list['Regn No']=='BR01PH4681']

,S. No,Select Demand Notice,Regn No,Office,Regn Date,Vehicle Class,Vehicle Category,purpose Code,Tax Type,Owner Name,...,Tax Arrear From,Tax Arrear UpTo,Permit Valid From,Permit Valid Upto,Tax Amount in (Rs),Tax Fine in (Rs),Total Tax Amount in (Rs),Fitness Validity,Financer Name,Blacklist Status


In [343]:
vt_tax_based_on[vt_tax_based_on['regn_no']=='BR01PA6073']['seat_cap']

310514     33
1119669    33
1236883    33
1903849    33
Name: seat_cap, dtype: int64

In [243]:
ft = pd.read_excel(r'C:\Users\CDMA\Documents\formula test.xlsx',sheet_name = 'Sheet1')
ft
#pd.read_sql_query(repr('Select '+ft['formula'].to_list()[0])[1:-1],con=engine)

,formula,result
0,<0> >10 AND <0> <=13 AND (<21>!=1) AND(<25>!=102),True


In [199]:
x = repr('Select '+ft['formula'].to_list()[0])[1:-1]
x.replace('')

"Select (10.0 >= 7 AND 10.0 <= 10 AND 2.0 != 1 AND 'nan' != 102) OR (10.0 >= 7 AND 10.0 <= 10 AND 0.0 IN(9,20))"

In [85]:
vt_tax[vt_tax['Regn No']=='GJ06AX5483']

,Regn No,Tax Mode,Payment Mode,Tax Amt,Tax Fine,Rcpt No,Rcpt Dt,Tax From,Tax Upto,Pur Cd,Flag,Collected By,State Cd,Off Cd
7699575,GJ06AX5483,Y,M,13527,399,RJ141R1804003371,2018-04-11 15:47:51.647,2018-03-26,2019-03-31,58,NaN,1612007479,RJ,141


In [269]:
calendar.isleap(2019)

False